In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./casia-train-model.ipynb
./README.md
./data/.gitignore
./data/Train/Train/Y_train.csv
./data/Train/Train/X_station_train.csv
./data/Train/Train/X_forecast/2D_arpege_20170214.nc
./data/Train/Train/X_forecast/arpege_3D_height_20170214.nc
./data/Train/Train/X_forecast/2D_arome_20170214.nc
./data/Train/Train/X_forecast/arpege_3D_isobar_20170214.nc
./data/Train/Train/Baselines/Baseline_forecast_train.csv
./data/Train/Train/Baselines/Baseline_observation_train.csv
./data/Test/Test/Id_month_test.csv
./data/Test/Test/X_station_test.csv
./data/Test/Test/X_forecast/arpege_3D_isobar_118.nc
./data/Test/Test/X_forecast/2D_arpege_119.nc
./data/Test/Test/X_forecast/arpege_3D_height_119.nc
./data/Test/Test/X_forecast/2D_arome_119.nc
./data/Test/Test/Baselines/Baseline_forecast_test.csv
./data/Test/Test/Baselines/Baseline_observation_test.csv
./data/Other/Other/NW_masks.nc
./data/Other/Other/stations_coordinates.csv
./.git/config
./.git/description
./.git/packed-refs
./.git/index
./.git/ORIG_HEAD
./.g

### load X_train

In [2]:
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)
coords_fname  = './data/Other/Other/stations_coordinates.csv'
fname = './data/Train/Train/X_station_train.csv'

In [19]:
def load_X_train(fname,coords_fname) :
    coords = pd.read_csv(coords_fname)
    train = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    # reorganisation des données.
    train['number_sta']=train['number_sta'].astype('category')
    #sort by station, then by date 
    train = train.sort_values(['number_sta','date'])
    method_traitements = ["mean","min","max","std"]
    dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                     ,"hu":method_traitements,"dd":method_traitements,"precip":"sum"}
    Base_obs = train[{"number_sta","date","ff","t","td","hu","dd","precip"}].copy()
    Base_obs.set_index('date',inplace = True)  

    # type de traitement.
    Base_obs_type = Base_obs.groupby('number_sta').resample('D').agg(dict_to_split)
    Base_obs_type.columns = ['_'.join(col).strip() for col in Base_obs_type.columns.values]
    Base_obs_type = Base_obs_type.reset_index(['date','number_sta'])
    Base_obs_type['number_sta'] = Base_obs_type['number_sta'].astype('category')   
    #Select the observations the day before

    Base_obs_type['Prediction'] = Base_obs_type['precip_sum'].shift(1)
    Base_obs_type = Base_obs_type.sort_values(by=["number_sta","date"])
    #del Base_obs['precip']
    #Base_obs = Base_obs.rename(columns={'baseline_obs':'precip'})
    del Base_obs_type["precip_sum"]
    date = first_date
    dates = []
    while date <= (last_date - datetime.timedelta(days=1)):
        date += datetime.timedelta(days=1)
        dates.append(date)

    d_dates =  pd.DataFrame(dates, columns = ['date'])
    d_dates['day_index'] = d_dates.index
    y_f = Base_obs_type.merge(d_dates,how="right",on = ["date"])
    y_f = y_f[y_f['date']!=last_date]
    y_f['Id'] = y_f[list(y_f.columns)[0]].astype(str) + '_' + \
                      y_f['day_index'].astype(str) 
    #y_f["number_sta"] = y_f[("number_sta","")]
    y_f = y_f.merge(coords,on=["number_sta"],how="right")
    X_train = y_f.copy()#drop(["day_index"],axis=1)
    X_train["month"] = X_train["date"].dt.month
    X_train["day"] = X_train["date"].dt.day
    X_train.drop(["date"],axis=1,inplace=True)
    return X_train

In [20]:
train = load_X_train(fname,coords_fname)

In [21]:
train

,number_sta,ff_mean,ff_min,ff_max,ff_std,t_mean,t_min,t_max,t_std,td_mean,...,dd_max,dd_std,Prediction,day_index,Id,lat,lon,height_sta,month,day
0,86118001,NaN,NaN,NaN,NaN,282.168750,279.80,284.55,1.266441,NaN,...,NaN,NaN,0.5,0.0,86118001_0,46.477,0.985,120.0,1.0,2.0
1,86118001,NaN,NaN,NaN,NaN,280.466667,278.85,282.05,0.792446,NaN,...,NaN,NaN,2.5,1.0,86118001_1,46.477,0.985,120.0,1.0,3.0
2,86118001,NaN,NaN,NaN,NaN,280.500000,280.15,280.90,0.350000,NaN,...,NaN,NaN,12.0,2.0,86118001_2,46.477,0.985,120.0,1.0,4.0
3,86149001,NaN,NaN,NaN,NaN,282.195833,278.70,284.90,1.794915,NaN,...,NaN,NaN,0.0,0.0,86149001_0,46.917,0.025,60.0,1.0,2.0
4,86149001,NaN,NaN,NaN,NaN,280.516667,276.60,282.55,1.548749,NaN,...,NaN,NaN,0.0,1.0,86149001_1,46.917,0.025,60.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183551,95690001,7.699167,4.50,12.28,1.966402,279.041667,277.38,280.63,1.008614,276.800000,...,259.0,28.574337,0.0,724.0,95690001_724,49.108,1.831,126.0,12.0,26.0
183552,95690001,8.134167,4.11,12.49,2.592839,277.274583,274.72,279.51,1.363101,275.160417,...,310.0,50.598161,2.4,725.0,95690001_725,49.108,1.831,126.0,12.0,27.0
183553,95690001,3.172500,1.29,6.96,1.459256,275.019583,273.10,278.44,1.570075,273.014583,...,295.0,18.572810,3.2,726.0,95690001_726,49.108,1.831,126.0,12.0,28.0
183554,95690001,6.341250,1.26,10.86,2.948950,277.452500,273.70,280.80,2.731512,275.341250,...,282.0,44.928929,0.0,727.0,95690001_727,49.108,1.831,126.0,12.0,29.0


### Load Y_test

In [165]:
fname_test = "./data/Test/Test/X_station_test.csv"
coords_fname_test = './data/Other/Other/stations_coordinates.csv'
coords = pd.read_csv(coords_fname_test)
test = pd.read_csv(fname_test)
test["Id"] =test["Id"].str.rsplit("_",n=1,expand=True)[0]
test.drop(["precip","month"],axis = 1,inplace = True)
method_traitements = ["mean","min","max","std"]
dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                 ,"hu":method_traitements,"dd":method_traitements}
test["Id"] = test["Id"].astype("category")
Base_obs = test[{"Id","ff","t","td","hu","dd"}].copy()
# type de traitement.
Base_obs_type = Base_obs.groupby('Id').agg(dict_to_split)
Base_obs_type = Base_obs_type.sort_values(by=["Id"])
test = Base_obs_type.copy()
test.columns = ['_'.join(col).strip() for col in test.columns.values]
test = test.reset_index(['Id'])
base_test = pd.read_csv("./data/Test/Test/Baselines/Baseline_observation_test.csv")
test = test.merge(base_test,on=["Id"],how="right")
test[["number_sta","day_index"]] = test["Id"].str.split("_",expand=True)
test["number_sta"] = test["number_sta"].astype(int)
test = test.merge(coords,on="number_sta",how='right')
date = first_date
dates = []
while date <= (last_date - datetime.timedelta(days=1)):
    date += datetime.timedelta(days=1)
    dates.append(date)

d_dates =  pd.DataFrame(dates, columns = ['date'])
d_dates['day_index'] = d_dates.index
d_dates["month"]=d_dates["date"].dt.month
d_dates["day"] = d_dates["date"].dt.day
del d_dates["date"]
test.dropna(subset=["day_index"],inplace=True)
test["day_index"] = test["day_index"].astype(int)
test = test.merge(d_dates,on=["day_index"],how="right")
test = test.dropna(subset=["lat"])

### Load Y

In [47]:
fname = './data/Train/Train/Y_train.csv'
param = 'Ground_truth'  #weather parameter name in the file ('Ground_truth' about Y and 'Prediction' about baseline)
ytrain= pd.read_csv(fname, parse_dates=['date'], infer_datetime_format=True)
ytrain['number_sta'] = ytrain['number_sta'].astype('category')

In [48]:
ytrain = ytrain[ytrain["date"]!=last_date].copy()

In [127]:
X_full = train.dropna()
Y_full = ytrain.dropna(subset=["Ground_truth"])
#X_full =X_full[X_full.index.isin(Y_full.index)].dropna()

In [128]:
X_full = X_full.sort_values("Id")

In [129]:
Y_full = Y_full.sort_values("Id")

In [130]:
value_list = list(X_full["Id"])
boolean_series = Y_full.Id.isin(value_list)
Y_full = Y_full[boolean_series]

In [131]:
value_list = list(Y_full["Id"])
boolean_series = X_full.Id.isin(value_list)
X_full = X_full[boolean_series]

In [135]:
Ximp = X_full.drop(["Id","number_sta","day_index","lon","lat","height_sta"],axis=1).values
Yimp = np.array(Y_full["Ground_truth"])

In [166]:
### pour l'ensemble test
x_full = test.dropna()

In [181]:
ximp = x_full.drop(["Id","number_sta","day_index","lon","lat","height_sta"],axis=1).values

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [50]:
from sklearn.model_selection import train_test_split

In [136]:
X_train, X_test, y_train, y_test = train_test_split(
...     Ximp, Yimp, test_size=0.33, random_state=42)

### Application du modèle.

In [21]:
from sklearn.impute import KNNImputer

In [22]:
### réaliser une imputation des données manquantes.

X = X_train.drop("Id",axis=1).values
imputer = KNNImputer(n_neighbors=2)
imputer.fit(X)

KNNImputer(n_neighbors=2)

In [174]:
from sklearn.impute import SimpleImputer

In [175]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [182]:
imp.fit(X_full.drop(["Id","number_sta","day_index","lon","lat","height_sta"],axis=1).values)

SimpleImputer()

In [184]:
ximp=imp.transform(test.drop(["Id","number_sta","day_index","lon","lat","height_sta"],axis=1).values)

### Fonction MAPE

In [137]:
def MAPE(yp,yt) :
    taille = len(yp)
    mape = 0
    for i in range(taille) :
        mape += abs((yt[i]-yp[i])/(yt[i]+1))
    mape *= 100/taille
    return mape

### Linear regression

In [68]:
from sklearn.linear_model import LinearRegression

In [141]:
reg = LinearRegression().fit(X_train, y_train)

In [142]:
reg.score(X_train, y_train)

0.23240890767167122

In [143]:
MAPE(reg.predict(X_train),y_train)

104.6640436654025

In [144]:
MAPE(reg.predict(X_test),y_test)

104.24008913023096

In [133]:
X_train.shape

(58097, 23)

### RandomForestRegression

In [57]:
from sklearn.ensemble import RandomForestRegressor

In [138]:
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

RandomForestRegressor()

In [139]:
regr.score(X_train,y_train)

0.9292559757991831

In [140]:
MAPE(regr.predict(X_train),y_train)

25.008112990761752

In [141]:
MAPE(regr.predict(X_test),y_test)

66.102658584232

In [146]:
train[train.index.isin(X_full.index)]["Id"]

17007       14066001_0
17008       14066001_1
17017      14066001_10
17107     14066001_100
17108     14066001_101
              ...     
183551    95690001_724
183552    95690001_725
183553    95690001_726
183554    95690001_727
183555    95690001_728
Name: Id, Length: 86712, dtype: object

In [186]:
y_predicted = list(regr.predict(ximp))
id_predicted = list(test["Id"])

In [187]:
df_submit = pd.DataFrame()
df_submit["Id"] = id_predicted
df_submit["Prediction"] = y_predicted

In [188]:
df_submit.to_csv('./working/' + "submission.csv",index=False)